In [53]:
import hmmlearn
import pybind11
import sys
import pandas as pd

print("hmmlearn version:", hmmlearn.__version__)
print("pybind11 version:", pybind11.__version__)
print("Python version:", sys.version)

hmmlearn version: 0.3.2
pybind11 version: 2.12.0
Python version: 3.9.15 (main, Nov 24 2022, 14:39:17) [MSC v.1916 64 bit (AMD64)]


In [54]:
import numpy as np
from hmmlearn import hmm


# Parameters

In [55]:
nWorkers = 5    # Number of Channels (labelers)
nSamples = 450   # Number of Data points
nClasses = 4    # Number of Classes

### Find HMM paramters from Ground-truth data

In [56]:
dataset_path = "../crowdsourced-datasets/my-dataset/multi_labeled_aligned_data.csv"

df = pd.read_csv(dataset_path).dropna(how='all')
df.columns
df_activity_dist = df.pivot_table(index='activpal_g_label', aggfunc='size').reset_index(name='counts')
df_activity_dist.columns = ['Class', 'Count']
df_activity_dist['Proportion'] = df_activity_dist['Count'] / df_activity_dist['Count'].sum()
print(df_activity_dist)
class_dict = df_activity_dist.set_index(df_activity_dist.index).to_dict()['Class']
startProb =np.array(df_activity_dist['Proportion'])

       Class   Count  Proportion
0    Cycling     246    0.001833
1    Running      76    0.000566
2      Still  122187    0.910342
3  Transport    5242    0.039055
4    Walking    6470    0.048204


### Find transition matrix (matrices)
The dataframe used here should be a small dataframe as a training

In [57]:
def compute_transition_matrices(df, timestamp_col='timestamp', participant_col='participant', label_col='activpal_g_label'):
    # Convert 'timestamp' column to datetime if it's not already
    df[timestamp_col] = pd.to_datetime(df[timestamp_col])

    # Step 1: Sort by participant and timestamp
    df = df.sort_values(by=[participant_col, timestamp_col]).reset_index(drop=True)

    # Step 2: Shift columns to get the next label and next timestamp for comparison
    df['next_label'] = df[label_col].shift(-1)
    df['next_timestamp'] = df[timestamp_col].shift(-1)
    df['next_participant'] = df[participant_col].shift(-1)
    # Step 2.5: Remove rows where either 'label_col' or 'next_label' is NaN
    df = df.dropna(subset=[label_col, 'next_label'])
    # Step 3: Filter for 1-minute transitions within the same participant
    transition_df = df[
        (df[participant_col] == df['next_participant']) &
        ((df['next_timestamp'] - df[timestamp_col]) == pd.Timedelta(minutes=1))
    ]

    # Step 4: Identify unique participants and classes
    participants = transition_df[participant_col].unique()
    classes = transition_df[label_col].unique()
    class_index = {cls: idx for idx, cls in enumerate(classes)}  # Map classes to indices

    # Initialize a dictionary to store each participant's transition matrix
    transition_matrices = {}

    for participant in participants:
        # Filter for current participant's transitions
        participant_transitions = transition_df[transition_df[participant_col] == participant]

        # Initialize a square matrix of size [num_classes x num_classes]
        matrix = np.zeros((len(classes), len(classes)), dtype=float)

        # Count transitions from each class to another
        for _, row in participant_transitions.iterrows():
            from_class = row[label_col]
            to_class = row['next_label']
            from_idx = class_index[from_class]
            to_idx = class_index[to_class]
            matrix[from_idx, to_idx] += 1

        # Normalize the matrix rows to get transition probabilities
        row_sums = matrix.sum(axis=1, keepdims=True)
        matrix = np.divide(matrix, row_sums, out=np.zeros_like(matrix), where=row_sums != 0)

        # Store the transition matrix for this participant
        transition_matrices[participant] = matrix

    return transition_matrices, class_index


In [58]:
# Assuming your DataFrame is called 'df'
transition_matrices, class_index = compute_transition_matrices(df)

In [59]:
def print_transition_matrix(matrix, class_index, title="Matrix"):
    """
    Prints the transition matrix beautifully, using class names.
    """
    # Convert the matrix to a DataFrame for pretty printing
    class_names = list(class_index.keys())
    matrix_df = pd.DataFrame(matrix, index=class_names, columns=class_names)
    
    # Print the formatted DataFrame
    print(f"{title}:\n")
    print(matrix_df.to_string(float_format="%.2f"))
    print("\n" + "="*50 + "\n")

# Printing a participant's transition matrix beautifully
for participant, matrix in transition_matrices.items():
    print(f"Transition matrix for participant {participant}:")
    print_transition_matrix(matrix, class_index)

Transition matrix for participant p1:
Matrix:

           Still  Walking  Transport  Cycling  Running
Still       0.95     0.05       0.00     0.00     0.00
Walking     0.52     0.47       0.01     0.00     0.00
Transport   0.01     0.01       0.98     0.00     0.00
Cycling     0.00     0.00       0.00     0.00     0.00
Running     0.00     0.00       0.00     0.00     0.00


Transition matrix for participant p10:
Matrix:

           Still  Walking  Transport  Cycling  Running
Still       0.99     0.01       0.00     0.00     0.00
Walking     0.61     0.37       0.02     0.00     0.00
Transport   0.03     0.04       0.93     0.00     0.00
Cycling     0.00     0.00       0.00     0.00     0.00
Running     0.00     0.00       0.00     0.00     0.00


Transition matrix for participant p11:
Matrix:

           Still  Walking  Transport  Cycling  Running
Still       0.96     0.04       0.00     0.00     0.00
Walking     0.49     0.48       0.02     0.00     0.00
Transport   0.01     0.03   

In [60]:
def average_transition_matrices(transition_matrices, class_index):
    """
    Averages the transition matrices for all participants, excluding zero rows.
    After averaging, each row is normalized to sum to 1 (probability distribution).
    """
    # Convert the transition matrices to a list of matrices
    all_matrices = list(transition_matrices.values())

    # Initialize a matrix to store the row-wise average
    num_classes = len(class_index)
    average_matrix = np.zeros((num_classes, num_classes), dtype=float)

    # Iterate over each row
    for row_idx in range(num_classes):
        # Collect non-zero rows for averaging
        non_zero_rows = [matrix[row_idx, :] for matrix in all_matrices if np.any(matrix[row_idx, :] != 0)]

        if non_zero_rows:
            # Calculate the row-wise average (exclude zero rows)
            row_average = np.mean(non_zero_rows, axis=0)
            # Normalize the row to sum to 1 (probability distribution)
            row_sum = row_average.sum()
            if row_sum != 0:
                row_average /= row_sum
            average_matrix[row_idx, :] = row_average

    return average_matrix

In [61]:

# Calculate the average transition matrix across all participants
average_matrix = average_transition_matrices(transition_matrices, class_index)
# Print the average matrix
print("Averaged Transition Matrix across all participants:")
print_transition_matrix(average_matrix, class_index, "Averaged Matrix")

Averaged Transition Matrix across all participants:
Averaged Matrix:

           Still  Walking  Transport  Cycling  Running
Still       0.98     0.02       0.00     0.00     0.00
Walking     0.48     0.48       0.04     0.00     0.00
Transport   0.02     0.02       0.95     0.00     0.00
Cycling     0.14     0.13       0.04     0.70     0.00
Running     0.00     0.40       0.00     0.00     0.60




In [62]:
def add_epsilon_and_normalize(matrix, epsilon=1e-6):
    """
    Adds a small epsilon to all elements in the matrix to ensure no zeros.
    Then normalizes each row so that each row sums to 1 (probability distribution).
    
    Parameters:
        matrix (np.array): The transition matrix to be modified.
        epsilon (float): The small value added to each element to avoid zeros.
    
    Returns:
        np.array: The matrix with epsilon added and rows normalized.
    """
    # Add epsilon to all elements
    matrix_with_epsilon = matrix + epsilon

    # Normalize each row so that the sum of elements in the row equals 1
    row_sums = matrix_with_epsilon.sum(axis=1, keepdims=True)
    normalized_matrix = matrix_with_epsilon / row_sums

    return normalized_matrix

In [63]:
# Add epsilon and normalize the average matrix
epsilon = 1e-4
average_matrix_with_epsilon = add_epsilon_and_normalize(average_matrix, epsilon)

# Print the final averaged matrix with epsilon added and normalized
print("Averaged Transition Matrix with Epsilon:")
print_transition_matrix(average_matrix_with_epsilon, class_index, "Averaged Matrix with Epsilon")

Averaged Transition Matrix with Epsilon:
Averaged Matrix with Epsilon:

           Still  Walking  Transport  Cycling  Running
Still       0.98     0.02       0.00     0.00     0.00
Walking     0.48     0.48       0.04     0.00     0.00
Transport   0.02     0.02       0.95     0.00     0.00
Cycling     0.14     0.13       0.04     0.70     0.00
Running     0.00     0.40       0.00     0.00     0.60




In [64]:
class_dict = {idx: class_name for  class_name,idx in class_index.items()}
# Reorder df_activity_dist based on class_index to align with the order of classes in class_index
# Reorder df_activity_dist based on the order of class_index values
df_activity_dist = df_activity_dist.set_index('Class').loc[
    [cls for cls in sorted(class_index, key=lambda x: class_index[x])]
].reset_index()

# Create the startProb array aligned with class_index order
startProb = np.array(df_activity_dist['Proportion'])

# Print reordered outputs for confirmation
print("\nReordered df_activity_dist:")
print(df_activity_dist)

print("\nReordered class_dict:")
print(class_dict)

print("\nStart probabilities (startProb):")
print(startProb)


Reordered df_activity_dist:
       Class   Count  Proportion
0      Still  122187    0.910342
1    Walking    6470    0.048204
2  Transport    5242    0.039055
3    Cycling     246    0.001833
4    Running      76    0.000566

Reordered class_dict:
{0: 'Still', 1: 'Walking', 2: 'Transport', 3: 'Cycling', 4: 'Running'}

Start probabilities (startProb):
[9.10341899e-01 4.82040813e-02 3.90549914e-02 1.83279815e-03
 5.66230322e-04]


# Parameters

In [65]:
def generate_emission_matrix(prob, size):
    """
    Generate an emission probability matrix with given diagonal probability.
    
    Args:
        prob (float): Probability for diagonal elements, must be between 0 and 1
        size (int): Size of the square matrix
        
    Returns:
        numpy.ndarray: Square matrix with diagonal elements = prob and other elements
                      containing equal portions of remaining probability
    
    Raises:
        ValueError: If prob is not between 0 and 1, or size is less than 1
    """
    if not 0 <= prob <= 1:
        raise ValueError("Probability must be between 0 and 1")
    if size < 1:
        raise ValueError("Size must be at least 1")
    
    # Calculate the remaining probability to distribute
    remaining_prob = 1 - prob
    
    # Calculate the probability for non-diagonal elements
    # For each row, we need to distribute remaining_prob across (size-1) elements
    other_prob = remaining_prob / (size - 1) if size > 1 else 0
    
    # Create matrix filled with other_prob
    matrix = np.full((size, size), other_prob)
    
    # Set diagonal elements to prob
    np.fill_diagonal(matrix, prob)
    
    return matrix

In [67]:

np.random.seed(42)
nClasses = len(class_dict)
model = hmm.CategoricalHMM(n_components=nClasses, n_features=nClasses)
model.startprob_ = startProb #np.array([0.6, 0.3, 0.1])
model.transmat_ = average_matrix_with_epsilon
model.emissionprob_ = generate_emission_matrix(0.9, nClasses)
# np.array([[0.7, 0.2, 0.1],
#                             [0.3, 0.5, 0.2],
#                             [0.3, 0.3, 0.4]])
# model.means_ = np.array([[0.0, 0.0], [3.0, -3.0], [5.0, 10.0]])
# model.covars_ = np.tile(np.identity(2), (3, 1, 1))
labeles, ground_truth = model.sample(1000)


In [4]:
remodel = hmm.GaussianHMM(n_components=3, covariance_type="full", n_iter=100)
remodel.fit(X)

GaussianHMM(covariance_type='full', n_components=3, n_iter=100)

In [5]:
nWorkers = 5    # Number of Channels
nSamples = 45   # Number of Data
nClasses = 4    # Number of Classes

In [8]:

# Number of possible observations
n_observations = 4

# Number of hidden states
n_states = 4

# Generate random observed data (replace with your actual data)
observations = np.random.randint(0, n_observations, size=1000)

# Create a Categorical Hidden Markov Model
model = hmm.CategoricalHMM(n_components=n_states, n_iter=100)

# Fit the model to the observations using the Baum-Welch algorithm
model.fit(observations.reshape(-1, 1))
obs=observations.reshape(-1, 1)


In [8]:
import numpy as np
from hmmlearn import hmm

# Generate synthetic data
np.random.seed(42)
X = np.concatenate([
    np.random.normal(0, 1, (100, 1)),
    np.random.normal(5, 1, (100, 1)),
    np.random.normal(10, 1, (100, 1))
])

# Initialize the Gaussian HMM
model = hmm.GaussianHMM(n_components=3, covariance_type="diag", n_iter=100, random_state=42)

# Fit the model to the data
model.fit(X)

# Predict the hidden states
hidden_states = model.predict(X)

# Print model parameters
print("Means:", model.means_)
print("Covariances:", model.covars_)
print("Transition Matrix:", model.transmat_)


Means: [[ 7.51580881]
 [ 7.57086256]
 [-0.10376863]]
Covariances: [[[7.53156374]]

 [[7.24896795]]

 [[0.81691181]]]
Transition Matrix: [[1.06136123e-03 9.98938639e-01 4.41129774e-19]
 [9.89463121e-01 1.05368787e-02 6.88686876e-19]
 [8.73411719e-03 1.26624615e-03 9.89999637e-01]]


In [16]:
from pomegranate import *
# state1 = State([DiscreteDistribution({'A': 0.5, 'B': 0.5}),
                # NormalDistribution(50, 10)], name='state1')
# NormalDistribution(50, 10)], name='state1'
d1 = DiscreteDistribution({'A' : 0.65, 'C' : 0.05, 'D' : 0.05, 'E' : 0.05, 'F' : 0.05, 'G' : 0.05, 'H' : 0.05, 'I' : 0.05})
d2 = DiscreteDistribution({'A' : 0.05, 'C' : 0.65, 'D' : 0.05, 'E' : 0.05, 'F' : 0.05, 'G' : 0.05, 'H' : 0.05, 'I' : 0.05})
d3 = DiscreteDistribution({'A' : 0.05, 'C' : 0.05, 'D' : 0.65, 'E' : 0.05, 'F' : 0.05, 'G' : 0.05, 'H' : 0.05, 'I' : 0.05})
d4 = DiscreteDistribution({'A' : 0.05, 'C' : 0.05, 'D' : 0.05, 'E' : 0.65, 'F' : 0.05, 'G' : 0.05, 'H' : 0.05, 'I' : 0.05})
d5 = DiscreteDistribution({'A' : 0.05, 'C' : 0.05, 'D' : 0.05, 'E' : 0.05, 'F' : 0.65, 'G' : 0.05, 'H' : 0.05, 'I' : 0.05})
d6 = DiscreteDistribution({'A' : 0.05, 'C' : 0.05, 'D' : 0.05, 'E' : 0.05, 'F' : 0.05, 'G' : 0.65, 'H' : 0.05, 'I' : 0.05})
d7 = DiscreteDistribution({'A' : 0.05, 'C' : 0.05, 'D' : 0.05, 'E' : 0.05, 'F' : 0.05, 'G' : 0.05, 'H' : 0.65, 'I' : 0.05})
d8 = DiscreteDistribution({'A' : 0.05, 'C' : 0.05, 'D' : 0.05, 'E' : 0.05, 'F' : 0.05, 'G' : 0.05, 'H' : 0.05, 'I' : 0.65})


s1 = State(d1, name="s1")
s2 = State(d2, name="s2")
s3 = State(d3, name="s3")
s4 = State(d4, name="s4")
s5 = State(d5, name="s5")
s6 = State(d6, name="s6")
s7 = State(d7, name="s7")
s8 = State(d8, name="s8")

model = HiddenMarkovModel('example')
model.add_states([s1, s2, s3, s4, s5, s6, s7, s8])
model.add_transition(model.start, s1, 0.90)
model.add_transition(model.start, s2, 0.10)
model.add_transition(s1, s1, 0.80)
model.add_transition(s1, s2, 0.20)
model.add_transition(s2, s2, 0.90)
model.add_transition(s2, s3, 0.10)
model.add_transition(s3, s3, 0.70)
model.add_transition(s3, model.end, 0.30)
model.bake()

print (model.log_probability(list('AAACDDDEEEGGHHHHIIII')))
-4.31828085576
print (", ".join(state.name for i, state in model.viterbi(list('AAACDDDEEEGGHHHHIIII'))[1]))


-52.67051117772867
example-start, s1, s1, s1, s2, s3, s3, s3, s3, s3, s3, s3, s3, s3, s3, s3, s3, s3, s3, s3, s3, example-end
